In [1]:
import os
import time
import threading
import webbrowser
from pathlib import Path
from typing import Tuple

from flask import Flask, request, jsonify


In [3]:

import JTBrix
from JTBrix import screen_config
from JTBrix.screen_config import flow_config
from JTBrix.ui.main import ui, submitted_results
from JTBrix.questionnaire.screens import screens


In [4]:

from JTBrix.utils.paths import get_project_paths
from JTBrix.utils.config import read_experiment_config
from JTBrix.utils.results import build_full_structured_result, get_combined_results
from JTBrix.io.saving import save_structured_output


In [6]:
from JTBrix.utils import find_free_port 
from JTBrix import  detect_environment

In [9]:
import os
import platform

print("System:", platform.system())

System: Darwin


In [8]:
sys   

'macOS'

In [ ]:
from JTBrix import run_test

In [ ]:
result = run_test("../src/JTBrix/data/config.yml", "../src/JTBrix/data/static/") 

In [ ]:
import os
import time
import threading
import webbrowser
from pathlib import Path
from typing import Tuple

from flask import Flask, request, jsonify

import JTBrix
from JTBrix import run_test, screen_config
from JTBrix.screen_config import flow_config
from JTBrix.ui.main import ui, submitted_results
from JTBrix.questionnaire.screens import screens
from JTBrix.experiment.run_experiment import run_test
from JTBrix.experiment.run_session import run_entire_test_config
from JTBrix.utils import find_free_port
from JTBrix.utils.paths import get_project_paths
from JTBrix.utils.config import read_experiment_config
from JTBrix.utils.results import build_full_structured_result, get_combined_results
from JTBrix.io.saving import save_structured_output

paths = get_project_paths()
template_path = paths["template_path"]
config_path = paths["config_path"]
static_path = paths["static_path"]
results_path = paths["results_path"]


start_timestamp = time.strftime("%Y-%m-%d %H:%M:%S")






def run_on_local():
    thread = threading.Thread(target=run_app)
    thread.daemon = True
    thread.start()
    print (f"Flask app running on port {port}")
    time.sleep(1)
    start_timestamp = time.strftime("%Y-%m-%d %H:%M:%S")
    webbrowser.open(f"http://127.0.0.1:{port}/experiment")

    
    # Wait until any submission is marked as finished
    print("Waiting for experiment to finish...")
    start_time = time.time()

    while time.time() - start_time < timeout:
        if any(entry.get("finished") for entry in submitted_results):
            break
        time.sleep(1)  # Check every second
    #thread.join()  # Or implement your own wait logic

    duration_seconds = int(time.time() - start_time)
    results = get_combined_results(submitted_results)
    results["experiment_start"] = start_timestamp
    results["experiment_duration_sec"] = duration_seconds

# NEW unified function
def run_test(config_path: str, static_folder: str, timeout: int = 600) -> Tuple[dict, list]:
    from flask import Flask
    import os, time, threading, webbrowser
    from pathlib import Path
    from JTBrix.ui.main import ui, submitted_results
    from JTBrix.questionnaire.screens import screens
    from JTBrix.utils import find_free_port
    from JTBrix.utils.config import read_experiment_config
    from JTBrix.utils.results import get_combined_results
    import JTBrix

    # Read config
    config, order = read_experiment_config(config_path)
    print("CONFIG:\n", config)
    print("\nSELECTED ORDER:", order)

    # Set up global config
    from JTBrix import screen_config
    screen_config.flow_config = config
    submitted_results.clear()

    # Setup paths
    jtbrix_root = Path(JTBrix.__file__).parent
    template_path = jtbrix_root / "templates"
    static_folder = os.path.abspath(static_folder)

    # Flask app
    app = Flask(__name__, static_folder=static_folder, template_folder=template_path)
    app.register_blueprint(ui)
    app.register_blueprint(screens)
    port = find_free_port()

    # Start server
    def run_app():
        app.run(port=port, debug=False, use_reloader=False)

    thread = threading.Thread(target=run_app)
    thread.daemon = True
    thread.start()

    print(f"Flask app running on port {port}")
    time.sleep(1)
    start_timestamp = time.strftime("%Y-%m-%d %H:%M:%S")
    webbrowser.open(f"http://127.0.0.1:{port}/experiment")

    # Wait for results
    print("Waiting for experiment to finish...")
    start_time = time.time()
    while time.time() - start_time < timeout:
        if any(entry.get("finished") for entry in submitted_results):
            break
        time.sleep(1)

    # Collect and return
    duration_seconds = int(time.time() - start_time)
    results = get_combined_results(submitted_results)
    results["experiment_start"] = start_timestamp
    results["experiment_duration_sec"] = duration_seconds
    return results, order


def run():
    results, order = run_test("../src/JTBrix/data/config.yml", "../src/JTBrix/data/static/", timeout=300) 
    print("Combined results:", results)
    print("Execution order:", order)

    structured_output = build_full_structured_result(results, "../src/JTBrix/data/config.yml", execution_order=order)
    print ("Structured output:", structured_output)
    print("Structured output keys:", structured_output.keys())
    print("Structured output values:", structured_output.values())




# Save to txt, yaml, csv, json
save_structured_output(structured_output, save_path="../src/JTBrix/data/results/", name="Test_data")
    

# ✅ Add /run_experiment route
@app.route("/run_experiment")
def run_experiment():
    results, order = run_test(config_path, static_path , timeout=300 )
    print("Waiting for experiment to finish...")
    start_time = time.time()
    structured_output = build_full_structured_result(results, config_path, execution_order=order)
    save_structured_output(structured_output, save_path=results_path, name="Test_data")
    return jsonify({"status": "success", "order": order, "summary": structured_output})

if __name__ == "__main__":
    port = find_free_port()
    # Create the app
    app = Flask(__name__, static_folder=static_path, template_folder=template_path)
    app.register_blueprint(ui)
    app.register_blueprint(screens)